In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import os
import re

In [ ]:
# Convert xlsx file to csv file
excel_files = ['decision.xlsx', 'perference.xlsx']
for excel_file in excel_files:
    df = pd.read_excel(excel_file)
    csv_file = excel_file.replace('.xlsx', '.csv')
    df.to_csv(csv_file, index=False, encoding='utf-8-sig')

In [ ]:
id_to_object = {
    1: 'PURPLE', 2: 'BLUE', 3: 'GREEN', 4: 'RED', 5: 'ORANGE',
    6: 'RISK', 7: 'HOPE', 8: 'SAFETY', 9: 'VITALITY', 10: 'POWER',
    11: 'LITERATURE', 12: 'PHYSICS', 13: 'MUSIC', 14: 'HISTORY', 15: 'GEOGRAPHY',
    16: 'SEA', 17: 'DESERT', 18: 'CITY', 19: 'MOUNTAIN', 20: 'VILLAGE'
}

answer_to_value = {
    'Very much': 4,
    'Somewhat': 3,
    'Not much': 2,
    'Not at all': 1
}

df = pd.read_csv('perference.csv', header=None, names=['user_id', 'question_id', 'answer'])

df['question_id'] = df['question_id'].map(id_to_object)
df['answer'] = df['answer'].map(answer_to_value)

pivot_df = df.pivot(index='user_id', columns='question_id', values='answer')
column_order = list(id_to_object.values())
pivot_df = pivot_df[column_order]

pivot_df.to_csv('perference_transformed.csv')

In [ ]:
# Define groups
groups = {
    'Colour': ['PURPLE', 'BLUE', 'GREEN', 'RED', 'ORANGE'],
    'Concept': ['RISK', 'HOPE', 'SAFETY', 'VITALITY', 'POWER'],
    'Discipline': ['LITERATURE', 'PHYSICS', 'MUSIC', 'HISTORY', 'GEOGRAPHY'],
    'Place': ['SEA', 'DESERT', 'CITY', 'MOUNTAIN', 'VILLAGE']
}

df = pd.read_excel('questions.xlsx', header=None)

questions = []
scores = []

for _, row in df.iterrows():
    q_num = row.iloc[0]
    match = re.findall(r'([A-Z]+) - (\d+), ([A-Z]+) - (\d+), ([A-Z]+) - (\d+), ([A-Z]+) - (\d+)', row.iloc[1])
    if match:
        g = match[0]
        questions.append({'Question': q_num, 'Colour': g[0], 'Concept': g[2], 'Discipline': g[4], 'Place': g[6]})
        scores.append({'Question': q_num, 'Colour': int(g[1]), 'Concept': int(g[3]), 'Discipline': int(g[5]), 'Place': int(g[7])})

questions = pd.DataFrame(questions)
scores = pd.DataFrame(scores)

In [ ]:
# Groups of objects
groups = {
    'colour': ['PURPLE', 'BLUE', 'GREEN', 'RED', 'ORANGE'],
    'concept': ['RISK', 'HOPE', 'SAFETY', 'VITALITY', 'POWER'],
    'discipline': ['LITERATURE', 'PHYSICS', 'MUSIC', 'HISTORY', 'GEOGRAPHY'],
    'place': ['SEA', 'DESERT', 'CITY', 'MOUNTAIN', 'VILLAGE']
}

files = ['2025 raw data.xlsx', '2024 raw data.xlsx', '2022 raw data.xlsx']
all_data_list = []

# Read and filter each file
for filename in files:
    df = pd.read_excel(filename)
    all_objects = [obj for sublist in groups.values() for obj in sublist]
    filtered_df = df[df['Answer'].isin(all_objects)].copy()
    all_data_list.append(filtered_df)

# Combine all years
combined_df = pd.concat(all_data_list, ignore_index=True)

plt.figure(figsize=(30, 5))

# Plot each group
for idx, (group_name, items) in enumerate(groups.items(), 1):
    plt.subplot(1, 4, idx)
    group_data = combined_df[combined_df['Answer'].isin(items)]
    counts = group_data['Answer'].value_counts()
    bars = plt.bar(range(len(counts)), counts.values)

    # Number labels
    for bar in bars:
        h = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2., h, f'{int(h)}', ha='center', va='bottom')

    plt.xticks(range(len(counts)), counts.index, rotation=45)
    plt.title(f'{group_name.title()} Group Distribution')
    plt.ylabel('Count')
    plt.ylim(0, 320)

plt.tight_layout()
plt.savefig("statistical_analysis_result/statistical_group_distribution.png", dpi=350)
plt.close()

# Print summary
print("\nSummary:")
print(f"Total samples: {len(combined_df)}")
print("\nCounts of each object:")
object_counts = combined_df['Answer'].value_counts()
print(object_counts)


In [ ]:
groups = {
    'colour': ['PURPLE', 'BLUE', 'GREEN', 'RED', 'ORANGE'],
    'concept': ['RISK', 'HOPE', 'SAFETY', 'VITALITY', 'POWER'],
    'discipline': ['LITERATURE', 'PHYSICS', 'MUSIC', 'HISTORY', 'GEOGRAPHY'],
    'place': ['SEA', 'DESERT', 'CITY', 'MOUNTAIN', 'VILLAGE']
}

# Read data
df = pd.read_csv('perference_transformed.csv', index_col=0)

# Create figure
fig, axes = plt.subplots(2, 2, figsize=(16, 13))
axes = axes.flatten()

# Violin plot for each group
for idx, (group_name, items) in enumerate(groups.items()):
    ax = axes[idx]
    available_columns = [item for item in items if item in df.columns]
    data_to_plot = [df[col].dropna().values for col in available_columns]

    parts = ax.violinplot(data_to_plot, positions=range(1, len(available_columns) + 1),
                          showmeans=True, showmedians=True, widths=0.7)

    for pc in parts['bodies']:
        pc.set_facecolor('#99CCFF')
        pc.set_alpha(0.7)
        pc.set_edgecolor('#333')
        pc.set_linewidth(1.2)

    if 'cmeans' in parts:
        parts['cmeans'].set_color('#FF4444')
        parts['cmeans'].set_linestyle('--')
        parts['cmeans'].set_linewidth(2)

    if 'cmedians' in parts:
        parts['cmedians'].set_color('#333')
        parts['cmedians'].set_linewidth(2)

    for k in ['cbars', 'cmins', 'cmaxes']:
        if k in parts:
            parts[k].set_color('#333')
            parts[k].set_linewidth(1.5)

    legend_elements = [
        Line2D([0], [0], color='#FF4444', linestyle='--', linewidth=2, label='Mean'),
        Line2D([0], [0], color='#333333', linewidth=2, label='Median')
    ]
    ax.legend(handles=legend_elements, loc='upper right', fontsize=10)
    ax.set_title(f'{group_name.title()} Group Distribution (Preference)', fontsize=14, fontweight='bold')
    ax.set_xlabel('Options', fontsize=12)
    ax.set_ylabel('Preference', fontsize=12)
    ax.set_ylim(0.5, 4.5)
    ax.set_xticks(range(1, len(available_columns) + 1))
    ax.set_xticklabels(available_columns, rotation=45)
    ax.grid(True, linestyle='--', alpha=0.3)

plt.tight_layout()
plt.savefig('statistical_analysis_result/preference_violin_plots.png', dpi=350, bbox_inches='tight')
plt.close()


In [ ]:
# Collect score data grouped by option
plot_data = {cat: {opt: [] for opt in opts} for cat, opts in groups.items()}

for _, q in questions.iterrows():
    s = scores[scores['Question'] == q['Question']].iloc[0]
    for cat in groups.keys():
        option = q[cat]
        score = s[cat]
        plot_data[cat][option].append(score)

# Plot violin figures
fig, axes = plt.subplots(2, 2, figsize=(16, 13))
axes = axes.flatten()

for idx, (category, options) in enumerate(groups.items()):
    ax = axes[idx]
    data = [plot_data[category][opt] for opt in options]

    parts = ax.violinplot(data, positions=range(1, len(options) + 1),
                          showmeans=True, showmedians=True, widths=0.8)

    for pc in parts['bodies']:
        pc.set_facecolor('#99CCFF')
        pc.set_alpha(0.7)
        pc.set_edgecolor('#333333')

    if 'cmeans' in parts:
        parts['cmeans'].set_color('#FF4444')
        parts['cmeans'].set_linestyle('--')
    if 'cmedians' in parts:
        parts['cmedians'].set_color('#333333')

    legend_elements = [
        Line2D([0], [0], color='#FF4444', linestyle='--', linewidth=2, label='Mean'),
        Line2D([0], [0], color='#333333', linewidth=2, label='Median')
    ]
    ax.legend(handles=legend_elements, loc='upper right', fontsize=10)

    ax.set_title(f'{category} Group Distribution (Score)', fontsize=14, fontweight='bold')
    ax.set_xlabel('Options', fontsize=12)
    ax.set_ylabel('Score', fontsize=12)
    ax.set_ylim(0.5, 4.5)
    ax.set_xticks(range(1, len(options) + 1))
    ax.set_xticklabels(options, rotation=45)
    ax.grid(True, linestyle='--', alpha=0.3)

plt.tight_layout()
plt.savefig('statistical_analysis_result/score_violin_plots.png', dpi=350, bbox_inches='tight')
plt.show()
